In [3]:
!pip install pdfplumber
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 53.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 78.0 MB/s eta 0:00:00:00:01


In [4]:
import os
import pdfplumber
from tqdm import tqdm
import tiktoken
from concurrent.futures import ThreadPoolExecutor, as_completed
import pickle
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer

In [5]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
def extract_text_from_pdf(pdf_path):

    text = list()
    with pdfplumber.open(pdf_path) as pdf:

        for page in pdf.pages:
            page_text = page.extract_text()

            if page_text:
                text.append(page_text)

    return "\n".join(text)

In [7]:
def load_all_resumes(single_dir_abs_path):

    documents = list()

    for root,_,files in os.walk(single_dir_abs_path):
        for file in files:
            if file.lower().endswith(".pdf"):
                pdf_path = os.path.join(root,file)
                text = extract_text_from_pdf(pdf_path)
                if text.strip():
                    documents.append(text)
    return documents

In [8]:
root_dir = "/kaggle/input/resume-dataset/data/data"

In [9]:
def process_resumes_per_category(single_dir):
    return single_dir, load_all_resumes(os.path.join(root_dir,single_dir))

In [10]:
"""
data_dict = dict()
with ThreadPoolExecutor(max_workers=os.cpu_count()) as pool:
    
    parallel_pools = [pool.submit(process_resumes_per_category, single_dir) for single_dir in os.listdir(root_dir)]
    for single_pool in tqdm(as_completed(parallel_pools), total=len(parallel_pools)):
        try:
            single_dir, resumes_raw_text_list = single_pool.result()
            data_dict[single_dir] = resumes_raw_text_list
        except Exception as e:
            print(f"Error processing {single_dir}: {e}")
"""

'\ndata_dict = dict()\nwith ThreadPoolExecutor(max_workers=os.cpu_count()) as pool:\n    \n    parallel_pools = [pool.submit(process_resumes_per_category, single_dir) for single_dir in os.listdir(root_dir)]\n    for single_pool in tqdm(as_completed(parallel_pools), total=len(parallel_pools)):\n        try:\n            single_dir, resumes_raw_text_list = single_pool.result()\n            data_dict[single_dir] = resumes_raw_text_list\n        except Exception as e:\n            print(f"Error processing {single_dir}: {e}")\n'

In [11]:
"""
gpt_tokenizer_encodings = tiktoken.get_encoding("o200k_base")
"""

'\ngpt_tokenizer_encodings = tiktoken.get_encoding("o200k_base")\n'

In [12]:
"""
with open("data_dict.pkl","wb") as file_handle:
    pickle.dump(data_dict,file_handle)
"""

'\nwith open("data_dict.pkl","wb") as file_handle:\n    pickle.dump(data_dict,file_handle)\n'

In [13]:
with open("/kaggle/input/proprocessed-data-pickle-file/data_dict.pkl","rb") as file_handle:
    data_dict = pickle.load(file_handle)

In [19]:
encoded_data_dict = {"Resume Encoded Text":[], "Suitable Job":[]}
row_idx = 0
max_len = 0

for k,v in data_dict.items():
    for resume_text in v:

        encoded_resume_text = tokenizer.tokenize(resume_text)
        encoded_resume_text = ["[CLS]"] + encoded_resume_text 
        encoded_data_dict["Resume Encoded Text"].append(encoded_resume_text)
        encoded_data_dict["Suitable Job"].append(k)

        if len(encoded_resume_text)+1 > max_len:
            max_len = len(encoded_resume_text)+1

In [20]:
bert_base_context_len = 512
max_len = max_len - (max_len % bert_base_context_len)
print(max_len)

6144


In [22]:
for idx, encoded_resume_text in enumerate(encoded_data_dict["Resume Encoded Text"]):
    encoded_data_dict["Resume Encoded Text"][idx] = encoded_resume_text + [0]*((max_len-1)-len(encoded_resume_text)) + ["[SEP]"]
    

In [23]:
data = pd.DataFrame(data=encoded_data_dict)

In [24]:
data.head()

,Resume Encoded Text,Suitable Job
0,"[[CLS], pre, -, press, graphic, designer, summ...",DESIGNER
1,"[[CLS], principle, designer, /, owner, profess...",DESIGNER
2,"[[CLS], project, designer, summary, team, -, o...",DESIGNER
3,"[[CLS], interior, designer, summary, a, result...",DESIGNER
4,"[[CLS], presentation, designer, summary, custo...",DESIGNER


In [25]:
data["Resume Encoded Text"] = data["Resume Encoded Text"].apply(lambda x: tokenizer.convert_tokens_to_ids(x))

In [27]:
data

,Resume Encoded Text,Suitable Job
0,"[101, 3653, 1011, 2811, 8425, 5859, 12654, 554...",DESIGNER
1,"[101, 6958, 5859, 1013, 3954, 2658, 12654, 459...",DESIGNER
2,"[101, 2622, 5859, 12654, 2136, 1011, 8048, 199...",DESIGNER
3,"[101, 4592, 5859, 12654, 1037, 3463, 8048, 585...",DESIGNER
4,"[101, 8312, 5859, 12654, 8013, 2326, 1998, 702...",DESIGNER
...,...,...
2478,"[101, 4007, 3992, 6337, 5281, 4007, 3992, 2236...",ARTS
2479,"[101, 5957, 16661, 2658, 6337, 29454, 29206, 2...",ARTS
2480,"[101, 5964, 1010, 6619, 1998, 7891, 2394, 3836...",ARTS
2481,"[101, 3772, 3694, 1018, 8013, 1013, 4435, 6337...",ARTS


In [28]:
shuffled_data = data.iloc[np.random.choice(np.arange(0,data.shape[0]),size=(data.shape[0],),replace=False)]

In [29]:
shuffled_data.reset_index(inplace=True,drop=True)

In [30]:
shuffled_data

,Resume Encoded Text,Suitable Job
0,"[101, 2343, 3237, 6337, 2865, 4262, 4013, 2007...",PUBLIC-RELATIONS
1,"[101, 23219, 5859, 2658, 12654, 2019, 23219, 2...",DESIGNER
2,"[101, 3095, 2075, 2449, 2458, 3208, 12654, 346...",BUSINESS-DEVELOPMENT
3,"[101, 3353, 8169, 2415, 3208, 17571, 5151, 292...",BANKING
4,"[101, 10516, 8325, 12654, 18114, 3167, 10365, ...",FITNESS
...,...,...
2478,"[101, 2270, 4262, 3353, 2658, 12654, 2026, 262...",PUBLIC-RELATIONS
2479,"[101, 4341, 12654, 17636, 4070, 3237, 2007, 20...",SALES
2480,"[101, 3026, 4070, 3208, 2658, 12654, 4813, 660...",PUBLIC-RELATIONS
2481,"[101, 8013, 2326, 8175, 12654, 6987, 1011, 804...",ADVOCATE


In [31]:
labels2idx = dict(zip(data_dict.keys(),range(0,len(data_dict.keys()))))

In [32]:
training_data = data.iloc[0:int(0.7*data.shape[0])]
testing_data = data.iloc[int(0.7*data.shape[0]):]

In [33]:
len(training_data.iloc[0,0])

6144

In [21]:
def training_data_generator(mb_size=79):

    for i in range(training_data.shape[0]//mb_size):

        X_mb = np.array(training_data.iloc[i*mb_size:(i+1)*mb_size,0])
        y_mb = np.array(training_data.iloc[i*mb_size:(i+1)*mb_size,1])

        yield X_mb, y_mb

In [ ]:
class BERTEncoderLayer(torch.nn.Module):

    def __init__(self):
        super().__init__()

        self.mha_layer = MHA() 

    def forward(self,emb_plus_pos_encoding):

        pass
        return encoder_embedding